In [164]:
import asyncio
import logging

from chromewhip import Chrome
from chromewhip.protocol import browser, page, dom
import lxml.html
import time
import datetime
import numpy as np
import pandas as pd
import time
from tqdm import tqdm_notebook as tqdm
from bs4 import BeautifulSoup
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
# see logging from chromewhip
logging.basicConfig(level=logging.WARN)

HOST = '127.0.0.1'
PORT = 9222

loop = asyncio.get_event_loop()
c = Chrome(host=HOST, port=PORT)

In [3]:
loop.run_until_complete(c.connect())

In [4]:
def sync_cmd(*args, **kwargs):
    return loop.run_until_complete(tab.send_command(*args, **kwargs))

In [48]:
name='apple'
tab = c.tabs[0]
tab = loop.run_until_complete(c.create_tab())
loop.run_until_complete(tab.enable_page_events())
result = sync_cmd(page.Page.navigate(url='https://www.owler.com/company/'+name), 
                  await_on_event_type=page.FrameStoppedLoadingEvent)
result = sync_cmd(dom.DOM.getDocument())
result = sync_cmd(dom.DOM.getOuterHTML(result['ack']['result']['root'].nodeId))
page_source = result['ack']['result']['outerHTML']
soup = BeautifulSoup(page_source, 'html.parser')
    
tmp_revenue_list = soup.find_all('div', attrs={'class': 'table-col revenue'})[1:]
competitors_revenue = []
for x in tmp_revenue_list:
    tmpvalue = re.findall(".*>(.*)<.*",str(x))[0]
    if len(tmpvalue)!=0:
        competitors_revenue.append(tmpvalue[1:])
competitors_revenue_float=[]
for i in competitors_revenue: 
    if i[:6] ==' &lt; ':
        competitors_revenue_float.append(convert_string_to_money(i[6:]))
    else:
        competitors_revenue_float.append(convert_string_to_money(i))
competitors_revenue_sum = np.array(competitors_revenue_float[1:]).sum()/10e6 #skip the searched company
try:
    annual_rev = competitors_revenue_float[0]/10e6
except:
    annual_rev = np.nan

competitor_count = len(competitors_revenue_float[1:])

HHI = (np.array(competitors_revenue_float)**2).sum()/((np.array(competitors_revenue_float).sum())**2)
print (name,annual_rev, competitor_count, competitors_revenue_sum, HHI)
loop.run_until_complete(c.close_tab(tab))

apple 265600.0 10 935200.0 0.13267737120400744


In [7]:
len(revenues)

5

In [22]:
def convert_string_to_money(string):
    if string[-1] =='M':
        return float(string[:-1])*10e6
    elif string[-1]=='B':
        return float(string[:-1])*10e9
    else:
        print ('Do not know how to convert ',string)

In [49]:
def crawl_owler_company_info(name):
    tab = c.tabs[0]
    tab = loop.run_until_complete(c.create_tab())
    loop.run_until_complete(tab.enable_page_events())
    company_url = 'https://www.owler.com/company/'+name
    print (company_url)
    result = sync_cmd(page.Page.navigate(url=company_url), 
                      await_on_event_type=page.FrameStoppedLoadingEvent)
    result = sync_cmd(dom.DOM.getDocument())
    result = sync_cmd(dom.DOM.getOuterHTML(result['ack']['result']['root'].nodeId))
    page_source = result['ack']['result']['outerHTML']
    soup = BeautifulSoup(page_source, 'html.parser')

    tmp_revenue_list = soup.find_all('div', attrs={'class': 'table-col revenue'})[1:]
    competitors_revenue = []
    for x in tmp_revenue_list:
        tmpvalue = re.findall(".*>(.*)<.*",str(x))[0]
        if len(tmpvalue)!=0:
            competitors_revenue.append(tmpvalue[1:])
    competitors_revenue_float=[]
    for i in competitors_revenue: 
        if i[:6] ==' &lt; ':
            competitors_revenue_float.append(convert_string_to_money(i[6:]))
        else:
            competitors_revenue_float.append(convert_string_to_money(i))
    competitors_revenue_sum = np.array(competitors_revenue_float[1:]).sum()/10e6 #skip the searched company
    try:
        annual_rev = competitors_revenue_float[0]/10e6
    except:
        annual_rev = np.nan

    competitor_count = len(competitors_revenue_float[1:])

    HHI = (np.array(competitors_revenue_float)**2).sum()/((np.array(competitors_revenue_float).sum())**2)
    print (name,annual_rev, competitor_count, competitors_revenue_sum, HHI)
    loop.run_until_complete(c.close_tab(tab))
    return annual_rev, competitor_count, competitors_revenue_sum, HHI

In [91]:
company_info_df = pd.read_excel('company_name.xlsx',index_col=0)

In [92]:
info_df =  pd.DataFrame(index=company_info_df.index ,columns=np.concatenate((np.array(['Company Business', 'Category', 'Company Revenue',
       'Competitor count (cap at 10)', 'Competitor Revenue in Total', 'HHI',
       'crawl_tried', 'potential crawl failing point', 'notes',
                  ]),np.array(['competitors revenue '+str(x+1) for x in range(10)]))))
info_df['crawl_tried'] =0
info_df['potential crawl failing point']=0
info_df['notes']=''
info_df.iloc[:,9:19] = 0

In [96]:
#info_df = pd.read_csv('tmp_company_info.csv',index_col=0)
for i in tqdm(range(info_df.shape[0])):
    if (info_df.iloc[i]['crawl_tried'] ==0) or (info_df.iloc[i]['potential crawl failing point'] ==1):
        try:
            info_df.iloc[i]['crawl_tried'] =1   
            #info_df.iloc[i,2:6] = crawl_owler_company_info(info_df.index[i])
            
            name=info_df.index[i]
            tab = c.tabs[0]
            tab = loop.run_until_complete(c.create_tab())
            loop.run_until_complete(tab.enable_page_events())
            result = sync_cmd(page.Page.navigate(url='https://www.owler.com/company/'+name), 
                              await_on_event_type=page.FrameStoppedLoadingEvent)
            result = sync_cmd(dom.DOM.getDocument())
            result = sync_cmd(dom.DOM.getOuterHTML(result['ack']['result']['root'].nodeId))
            page_source = result['ack']['result']['outerHTML']
            soup = BeautifulSoup(page_source, 'html.parser')
            
            
            tmp_revenue_list = soup.find_all('div', attrs={'class': 'table-col revenue'})[1:]
            competitors_revenue = []
            for x in tmp_revenue_list:
                tmpvalue = re.findall(".*>(.*)<.*",str(x))[0]
                if len(tmpvalue)!=0:
                    competitors_revenue.append(tmpvalue[1:])
            competitors_revenue_float=[]
            for ixx in competitors_revenue: 
                if ixx[:6] ==' &lt; ':
                    competitors_revenue_float.append(convert_string_to_money(ixx[6:]))
                else:
                    competitors_revenue_float.append(convert_string_to_money(ixx))
            competitors_revenue_sum = np.array(competitors_revenue_float[1:]).sum()/10e6 #skip the searched company
            try:
                annual_rev = competitors_revenue_float[0]/10e6
            except:
                annual_rev = np.nan

            competitor_count = len(competitors_revenue_float[1:])

            HHI = (np.array(competitors_revenue_float)**2).sum()/((np.array(competitors_revenue_float).sum())**2)
            print (name,annual_rev, competitor_count, competitors_revenue_sum, HHI)
            info_df.iloc[i,2:6] = annual_rev, competitor_count, competitors_revenue_sum, HHI
            info_df.iloc[i,9:competitor_count+9] = competitors_revenue_float[1:]
            loop.run_until_complete(c.close_tab(tab))
            
            
        except:
            info_df.iloc[i]['notes']='connectionclosed'
            #loop.run_until_complete(c.close_tab(tab))
        info_df.iloc[i]['crawl_tried'] =1
        if i>10:
            if info_df.iloc[i-10:i]['Competitor Revenue in Total'].sum()==0:
                print('the crwaler may have failed')
                info_df.iloc[i-10:i]['potential crawl failing point']=1
        if info_df.iloc[i]['Competitor Revenue in Total']!=0:
            info_df.iloc[i]['potential crawl failing point']=0
        info_df.to_csv('/home/chenxupeng/projects/crwal_company_info/tmp_company_info.csv')
        time.sleep(10)

HBox(children=(IntProgress(value=0, max=1414), HTML(value='')))

/home/shibinbin/apps/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/shibinbin/apps/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars


buildingrobotics nan 0 0.0 nan
medicalsimulation nan 0 0.0 nan
cancergenetics 24.2 10 635.1 0.42384963125104835


/home/shibinbin/apps/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


kindredbiosciences nan 0 0.0 nan
topgifter 2.6 6 82.5 0.3078579013284986
itnenergysystems nan 0 0.0 nan
pica9 7.0 10 214.8 0.17687077042413046
livelyformerlyhamlet nan 0 0.0 nan
actionx 1.0 8 3477.9 0.47896876860716786
forgerock 73.8 10 388.3 0.15601381516154553
nakivo 19.0 10 6798.4 0.5231535939348865
bobler nan 0 0.0 nan
smartenergyinstruments nan 0 0.0 nan
cytogelpharma 43.5 4 719.3 0.8318307939534444
invine nan 0 0.0 nan
shake nan 0 0.0 nan
firstfunder nan 0 0.0 nan
regulatorybinder nan 0 0.0 nan
stroho nan 0 0.0 nan
outboundengine 31.2 10 499.9 0.15895498806826927
fidelissecuritysystems nan 0 0.0 nan
capzure 1.0 1 3.9 0.6751353602665556


...

hopin nan 0 0.0 nan
kairoswatches 1.0 6 187.2 0.8784293508274034
ultradia nan 0 0.0 nan
cooperatize nan 0 0.0 nan
wishinguwell nan 0 0.0 nan
lateralsv nan 0 0.0 nan
rsvplaw nan 0 0.0 nan
teamblind 1.1 10 2882.4 0.3393911923731254
vibby nan 0 0.0 nan
zipier nan 0 0.0 nan
cellagain nan 0 0.0 nan
jisto 1.0 10 93616.2 0.2601885011713582
fashiongps 2.7 4 60.0 0.3860997280790784
flashmit nan 0 0.0 nan
foodcrave 4.0 10 99.6 0.2188100952579717
teleretailcorporation nan 0 0.0 nan
urbanbound 5.3 10 9053.7 0.4883652599318829
gogig nan 0 0.0 nan
waypointmedia 4.4 9 113.8 0.17697355424429043
autolotto 1.5 4 17.5 0.2742382271468144
flyspansystems nan 0 0.0 nan
fusion3design nan 0 0.0 nan
autopilot2 nan 0 0.0 nan
linguistiko nan 0 0.0 nan
creativwire nan 0 0.0 nan
icecreamlabs 1.0 9 110.9 0.2038931095921371
tenantturner 1.0 10 1838.8 0.3448552101962713
thecounterrhythmgroup nan 0 0.0 nan
joto nan 0 0.0 nan
monkey 1.0 10 2045.8 0.3685353408375726
logfuze 1.0 10 4002.6 0.46241344125086103
questis nan 0

ERROR:chromewhip.chrome.ChromeTab.send_handler:{"method": "Page.navigate", "params": {"url": "https://www.owler.com/company/classwork"}, "id": 2}
ERROR:chromewhip.chrome.ChromeTab.send_handler:{"method": "Page.navigate", "params": {"url": "https://www.owler.com/company/herringbone"}, "id": 2}


inappad nan 0 0.0 nan
wardrobewakeup nan 0 0.0 nan
bestimators nan 0 0.0 nan
silverwing nan 0 0.0 nan
moneylion 3.7 10 251.6 0.4440975487314832
ontopic nan 0 0.0 nan
zumaventures nan 0 0.0 nan
avox2 nan 0 0.0 nan
keanr nan 0 0.0 nan
bentpixels nan 0 0.0 nan
trinitypharmasolutions nan 0 0.0 nan
advancedwearabledevices nan 0 0.0 nan
parionsciences nan 0 0.0 nan
ivincipartners nan 0 0.0 nan
moneero nan 0 0.0 nan
proteamedical nan 0 0.0 nan
the crwaler may have failed
lacoonsecurity nan 0 0.0 nan
the crwaler may have failed
airtouchcommunications nan 0 0.0 nan
the crwaler may have failed
solidinteractive nan 0 0.0 nan
the crwaler may have failed
graybug 28.6 9 38.6 0.2356283659297052
the crwaler may have failed
highperformancesystemscorporation nan 0 0.0 nan
mongoosemetrics 1.0 9 197.3 0.3054538972084707
nventi 5.7 7 105.1 0.2290138018219969
supplychainconnect nan 0 0.0 nan


ERROR:chromewhip.chrome.ChromeTab.send_handler:{"method": "Page.navigate", "params": {"url": "https://www.owler.com/company/bioconsortia"}, "id": 2}


colonialbankfsb 17.9 10 913.8 0.7151744496888938
livongohealth nan 0 0.0 nan
paracosm 1.2 9 77.2 0.3343821584756351
extremeplasticsplus 5.0 3 34.0 0.34911242603550297
pronutria nan 0 0.0 nan


In [128]:
name='google'
tab = c.tabs[0]
tab = loop.run_until_complete(c.create_tab())
loop.run_until_complete(tab.enable_page_events())
result = sync_cmd(page.Page.navigate(url='https://www.crunchbase.com/organization/'+name), 
                  await_on_event_type=page.FrameStoppedLoadingEvent)
result = sync_cmd(dom.DOM.getDocument())
result = sync_cmd(dom.DOM.getOuterHTML(result['ack']['result']['root'].nodeId))
page_source = result['ack']['result']['outerHTML']
tree = lxml.html.fromstring(page_source)
loop.run_until_complete(c.close_tab(tab))

In [129]:
xpath_revenue = '//*[@id="section-overview"]/mat-card/div[2]/image-with-fields-card/image-with-text-card/div/div/div[2]/div[2]/field-formatter/span/text()'

company_intro = tree.xpath(xpath_revenue)[0]
company_intro

'Google is a multinational corporation that is specialized in internet-related services and products.'

In [167]:
company_categories = ''
for t in range(1,20):
    xpath_revenue = '//*[@id="section-overview"]/mat-card/div[2]/fields-card[1]/div/span[2]/field-formatter/identifier-multi-formatter/span/a['+str(t)+']'
    
    if tree.xpath(xpath_revenue)==[]:
        break
    else:
        company_categories = company_categories+', '+tree.xpath(xpath_revenue)[0].text[1:]
company_categories[2:]

'Biotechnology, Health Care, Hospital'

In [140]:
crunch_info_df = pd.read_excel('crunch_company_name.xlsx',index_col=0)

In [142]:
info_df =  pd.DataFrame(index=crunch_info_df.index ,columns=['Company Business', 'Category'
       'crawl_tried', 'potential crawl failing point', 'notes'])
info_df['crawl_tried'] =0
info_df['notes']=''

In [ ]:
#info_df = pd.read_csv('tmp_company_info.csv',index_col=0)
for i in tqdm(range(info_df.shape[0])):
    if info_df.iloc[i]['crawl_tried'] ==0 :
        try:
            info_df.iloc[i]['crawl_tried'] =1   

            name=info_df.index[i]
            
            tab = c.tabs[0]
            tab = loop.run_until_complete(c.create_tab())
            loop.run_until_complete(tab.enable_page_events())
            result = sync_cmd(page.Page.navigate(url='https://www.crunchbase.com/organization/'+name), 
                              await_on_event_type=page.FrameStoppedLoadingEvent)
            result = sync_cmd(dom.DOM.getDocument())
            result = sync_cmd(dom.DOM.getOuterHTML(result['ack']['result']['root'].nodeId))
            page_source = result['ack']['result']['outerHTML']
            tree = lxml.html.fromstring(page_source)
            loop.run_until_complete(c.close_tab(tab))
            xpath_revenue = '//*[@id="section-overview"]/mat-card/div[2]/image-with-fields-card/image-with-text-card/div/div/div[2]/div[2]/field-formatter/span/text()'

            company_intro = tree.xpath(xpath_revenue)[0]
            company_categories = ''
            for t in range(1,20):
                xpath_revenue = '//*[@id="section-overview"]/mat-card/div[2]/fields-card[1]/div/span[2]/field-formatter/identifier-multi-formatter/span/a['+str(t)+']'

                if tree.xpath(xpath_revenue)==[]:
                    break
                else:
                    company_categories = company_categories+', '+tree.xpath(xpath_revenue)[0].text[1:]
            
            info_df.iloc[i,0] = company_intro
            info_df.iloc[i,1] = company_categories[2:]
            print (name,company_intro,company_categories[2:])
            
            loop.run_until_complete(c.close_tab(tab))
            
            
        except:
            info_df.iloc[i]['notes']='connectionclosed'
            #loop.run_until_complete(c.close_tab(tab))
        info_df.iloc[i]['crawl_tried'] =1
        
        info_df.to_csv('/home/chenxupeng/projects/crwal_company_info/tmp_company_info_crunch.csv')
        time.sleep(np.random.uniform(1.5,2))

HBox(children=(IntProgress(value=0, max=1414), HTML(value='')))

ERROR:chromewhip.chrome.ChromeTab.send_handler:{"method": "Page.navigate", "params": {"url": "https://www.crunchbase.com/organization/kindred-biosciences"}, "id": 2}
ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<ChromeTab.send_command() done, defined at /home/shibinbin/apps/anaconda3/lib/python3.6/site-packages/chromewhip/chrome.py:279> exception=TimeoutError('Unknown cause for timeout to occurs for "Page.navigate" with id=2',)>
Traceback (most recent call last):
  File "/home/shibinbin/apps/anaconda3/lib/python3.6/site-packages/chromewhip/chrome.py", line 180, in _send
    await asyncio.wait_for(ack_event.wait(), timeout=TIMEOUT_S)  # recv
  File "/home/shibinbin/apps/anaconda3/lib/python3.6/asyncio/tasks.py", line 362, in wait_for
    raise futures.TimeoutError()
concurrent.futures._base.TimeoutError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/shibinbin/apps/anaconda3/lib/python

building-robotics Comfy is changing the way people interact with their workplaces. Formerly known as Building Robotics. Artificial Intelligence, Business Intelligence, Enterprise Software, Mobile Apps, SaaS, Smart Building
medical-simulation Medical Simulation provides full-service simulation training and consulting services for healthcare personnel and medical societies. Biotechnology, Health Care, Hospital


ERROR:chromewhip.chrome.ChromeTab.send_handler:{"method": "Page.navigate", "params": {"url": "https://www.crunchbase.com/organization/cancer-genetics"}, "id": 2}
ERROR:chromewhip.chrome.ChromeTab.send_handler:{"method": "Page.navigate", "params": {"url": "https://www.crunchbase.com/organization/kindred-biosciences"}, "id": 2}


topgifter Search and get Gift Recommendations E-Commerce, Gift, Shopping
itn-energy-systems ITN Energy Systems is a research and product development organization focused specifically on technology development and commercialization. Information Technology, Product Management, Product Research
pica9--inc- The leading SaaS Brand Management Platform empowering brands to deliver local campaigns at scale. Advertising, Brand Marketing, Digital Media, Direct Marketing, Enterprise Software, Local Advertising, Marketing Automation, SaaS
lively-formerly-hamlet Lively is the connected home health platform that provides a medical alert device older adults actually want to wear Elderly, Hardware, Internet of Things, mHealth, Software
actionx ActionX connects gaps between mobile customer acquisition and mobile retargeting to drive transactions and improve customer lifetime value. Ad Retargeting, Mobile
forgerock ForgeRock digital identity management solutions transform how organizations securely inte

...

ERROR:chromewhip.chrome.ChromeTab.send_handler:{"method": "Page.navigate", "params": {"url": "https://www.crunchbase.com/organization/ripple-labs"}, "id": 2}


fivetran Fivetran technology helps analysts replicate data into a cloud warehouse. Analytics, Big Data, Cloud Data Services, Data Integration, Enterprise Software, Warehousing
kinsa-inc Kinsa is creating the world's first real-time map of human health. Child Care, Fitness, Health Care, mHealth
opendorse The athlete marketing platform. Thousands of athletes use opendorse to share content on social.  Brand Marketing, Enterprise Software, Management Consulting, Publishing, Service Industry, Social Media Management, Sports
near-infinity World leader in displacement monitoring services with satellite radar data Cloud Computing, Computer, Software
collabra Collabra is a platform that provides tools to reinforce their music lessons, capture progress, and create accountability. Enterprise Software, Mobile, Music, SaaS
dropifi Dropifi provides a customized smart widget that allows companies to analyze, discover and re-channel messages. Customer Service, Social Media, Social Media Management, So

talkto TalkTo is a smartphone app that helps its users correspond with local businesses via text messages. Information Technology, Messaging, Mobile Apps
armagen-technologies ArmaGen is focused on developing technologies for siRNA delivery across the blood-brain barrier. Biotechnology, Health Care, Medical
datarobot DataRobot provides a predictive analytics platform to rapidly build and deploy predictive models the cloud or an enterprise. Artificial Intelligence, Enterprise Software, Information Technology, Machine Learning, Predictive Analytics, SaaS
qlika Qlika enables advertisers to manage and optimize their campaigns for each local market, segment and media channel. Advertising, Analytics, Local
survata Survata is a marketing research company that sells brand intelligence to the world’s largest brands. Advertising, Analytics, Consumer Research, Market Research, Product Research, Test and Measurement
vionic Vionic is a social media deals platform that enables brands to offer exclusi

ERROR:chromewhip.chrome.ChromeTab.send_handler:{"method": "Page.navigate", "params": {"url": "https://www.crunchbase.com/organization/hanesbrands"}, "id": 2}


tapblaze TapBlaze is our 1st company division which caters to casual gamers. It is our committed duty to deliver a Happy experience to our customers. Apps, Mobile
tagapet TagaPet develops pet protection products that incorporate QR code and NFC technology. Mobile, NFC
discovery-bay-games Discovery Bay Games is a creator of family-oriented and educational multi-player games. Consumer, Gaming
scratch-hard Scratch Hard is a shopping companion application for browsing in-store sales and promotions, saving coupons, and sharing user experience. Coupons, Fashion, Finance, Mobile, Retail, Shopping
litpick LitPick is a place for young adults to read, review, and share the latest in young adult literature. Communities, EdTech, Education
